# 쿨루프 시공 대상 여부 분류

# 1. Data Preprocessing

In [1]:
!pip install keras --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.2.1 which is incompatible.


In [2]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [3]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

import os
import random
from PIL import Image

from sklearn.model_selection import train_test_split

import keras

import shutil

### 1) 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [4]:
def dataset_extract(file_name):
    # 파일 경로에서 확장자를 제외한 기본 이름 추출
    base_name = os.path.basename(file_name)
    folder_name = os.path.splitext(base_name)[0]

    # 추출된 폴더 경로 설정
    extract_folder_path = f'/content/{folder_name}/'

    with zipfile.ZipFile(file_name, 'r') as zip_ref:
        file_list = zip_ref.namelist()

        # 폴더가 이미 존재하는지 확인
        if os.path.exists(extract_folder_path):
            print(f'데이터셋 폴더가 이미 존재합니다: {extract_folder_path}')
            return
        else:
            # 폴더가 존재하지 않으면 파일 추출
            for f in tqdm(file_list, desc='Extracting', unit='files'):
                zip_ref.extract(member=f, path=extract_folder_path)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
dataset_extract('/content/drive/MyDrive/Colab Notebooks/KT_MP_4_2/cool_roof_images.zip')

Extracting: 100%|██████████| 200/200 [00:04<00:00, 45.86files/s]


In [7]:
dataset_extract('/content/drive/MyDrive/Colab Notebooks/KT_MP_4_2/cool_roof_yolo_labels.zip')

Extracting: 100%|██████████| 204/204 [00:00<00:00, 294.79files/s]


### 2) 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식에 맞춰 폴더 생성

In [8]:
# yolov8/data 폴더 확인 및 생성
yolov8_data_dir = 'yolov8/data'
os.makedirs(yolov8_data_dir, exist_ok=True)

In [9]:
# 구조에 맞는 디렉토리 생성
!mkdir /content/yolov8/data/images;
!mkdir /content/yolov8/data/images/train; mkdir /content/yolov8/data/images/val

!mkdir /content/yolov8/data/labels;
!mkdir /content/yolov8/data/labels/train; mkdir /content/yolov8/data/labels/val

### 3) Data Augmentation

데이터 증강

In [17]:
import os
from PIL import Image

import math

def rotate_point(x, y, angle, cx, cy):
    """
    주어진 점 (x, y)를 (cx, cy)를 중심으로 angle도 회전시킨 후의 좌표를 반환.
    """
    angle_rad = math.radians(angle)
    x_new = cx + math.cos(angle_rad) * (x - cx) - math.sin(angle_rad) * (y - cy)
    y_new = cy + math.sin(angle_rad) * (x - cx) + math.cos(angle_rad) * (y - cy)
    return x_new, y_new

def rotate_bbox(bbox, angle, image_width, image_height):
    """
    바운딩 박스를 주어진 각도만큼 회전시키는 함수.
    """
    x_center, y_center, width, height = bbox
    cx, cy = x_center * image_width, y_center * image_height  # 절대 좌표로 변환

    # 바운딩 박스의 네 꼭짓점 계산
    points = [
        (cx - width * image_width / 2, cy - height * image_height / 2),  # 좌상단
        (cx + width * image_width / 2, cy - height * image_height / 2),  # 우상단
        (cx - width * image_width / 2, cy + height * image_height / 2),  # 좌하단
        (cx + width * image_width / 2, cy + height * image_height / 2)   # 우하단
    ]

    # 각 점 회전
    rotated_points = [rotate_point(x, y, angle, cx, cy) for x, y in points]

    # 회전된 점들을 포함하는 최소 바운딩 박스 계산
    min_x = min(x for x, y in rotated_points)
    max_x = max(x for x, y in rotated_points)
    min_y = min(y for x, y in rotated_points)
    max_y = max(y for x, y in rotated_points)

    # 새로운 바운딩 박스의 중심, 너비, 높이 계산
    x_center_new = (min_x + max_x) / 2 / image_width
    y_center_new = (min_y + max_y) / 2 / image_height
    width_new = (max_x - min_x) / image_width
    height_new = (max_y - min_y) / image_height

    # 새로운 바운딩 박스 반환
    bbox = (x_center_new, y_center_new, width_new, height_new)
    return bbox


def process_image_and_bbox(image_path, bbox_file_path, output_image_path, output_bbox_path, angle):
    # 이미지 회전 및 저장
    image = Image.open(image_path)
    rotated_image = image.rotate(angle, expand=True)
    rotated_image.save(output_image_path)
    image_width, image_height = rotated_image.size

    # 바운딩 박스 파일 처리
    new_bbox_lines = []
    with open(bbox_file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            class_id = parts[0]
            bbox = tuple(map(float, parts[1:]))
            rotated_bbox = rotate_bbox(bbox, angle, image_width, image_height)

            # 소수점 아래 6자리로 제한하여 문자열 포맷팅
            new_bbox_line = f"{class_id} {' '.join(format(coord, '.6f') for coord in rotated_bbox)}\n"
            new_bbox_lines.append(new_bbox_line)

    # 새 바운딩 박스 파일 저장
    with open(output_bbox_path, 'w') as file:
        file.writelines(new_bbox_lines)


# 경로 설정
image_folder = '/content/cool_roof_images'
bbox_folder = '/content/cool_roof_yolo_labels/obj_train_data'
output_image_folder = 'rotated_img_30'
output_bbox_folder = 'rotated_label_30'
angle = 30

# 폴더 생성
os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(output_bbox_folder, exist_ok=True)

# 모든 이미지 처리
for image_name in os.listdir(image_folder):
    if not image_name.endswith('.jpg'):
        continue
    base_name = os.path.splitext(image_name)[0]

    image_path = os.path.join(image_folder, image_name)
    bbox_file_path = os.path.join(bbox_folder, base_name + '.txt')
    output_image_path = os.path.join(output_image_folder, base_name + '_rotated_30.jpg')
    output_bbox_path = os.path.join(output_bbox_folder, base_name + '_rotated_30.txt')

    process_image_and_bbox(image_path, bbox_file_path, output_image_path, output_bbox_path, angle)


증강한 이미지, 라벨 데이터들 모두 각 폴더에 저장

In [18]:
# 원본 및 회전된 이미지가 저장된 폴더 목록
source_folders = [
    '/content/cool_roof_images',
    '/content/rotated_img_10',
    '/content/rotated_img_20',
    '/content/rotated_img_30',
    '/content/rotated_img_40'
]

# 모든 이미지를 저장할 폴더
output_folder = 'cool_roof_images_all'
os.makedirs(output_folder, exist_ok=True)

# 각 폴더에서 이미지를 읽어서 img_all 폴더로 복사
for folder in source_folders:
    for image_name in os.listdir(folder):
        if image_name.endswith('.jpg'):
            source_path = os.path.join(folder, image_name)
            destination_path = os.path.join(output_folder, image_name)
            shutil.copy(source_path, destination_path)

print(f" '{output_folder}'에 모든 이미지 저장 완료")

 'cool_roof_images_all'에 모든 이미지 저장 완료


In [19]:
# 원본 및 회전된 바운딩 박스 정보가 저장된 폴더 목록
source_folders = [
    '/content/cool_roof_yolo_labels/obj_train_data',
    '/content/rotated_label_10',
    '/content/rotated_label_20',
    '/content/rotated_label_30',
    '/content/rotated_label_40'
]

# 모든 .txt 파일을 저장할 폴더
output_folder = 'cool_roof_labels_all'
os.makedirs(output_folder, exist_ok=True)

# 각 폴더에서 .txt 파일을 읽어서 label_all 폴더로 이동
for folder in source_folders:
    for txt_name in os.listdir(folder):
        if txt_name.endswith('.txt'):
            source_path = os.path.join(folder, txt_name)
            destination_path = os.path.join(output_folder, txt_name)
            # 파일 이동
            shutil.move(source_path, destination_path)

print(f" '{output_folder}'에 모든 라벨 저장 완료")

 'cool_roof_labels_all'에 모든 라벨 저장 완료


### 4) 데이터 split & 파일 이동
- train dataset : validation dataset = 80 : 20

In [20]:
# 기본 경로 설정
img_dir = '/content/cool_roof_images_all'  # 모든 이미지 폴더
label_dir = '/content/cool_roof_labels_all'  # 모든 레이블 폴더

# 분할 후 이미지 및 레이블 디렉토리 경로
img_train_dir = '/content/yolov8/data/images/train'
img_val_dir = '/content/yolov8/data/images/val'
label_train_dir = '/content/yolov8/data/labels/train'
label_val_dir = '/content/yolov8/data/labels/val'


# 이미지 파일 목록 로드 및 분할
img_files = [f for f in os.listdir(img_dir) if f.endswith('.jpg')]
train_img_files, val_img_files = train_test_split(img_files, test_size=0.2, random_state=2024)

# 이미지 이동
for f in train_img_files:
    shutil.move(os.path.join(img_dir, f), img_train_dir)

for f in val_img_files:
    shutil.move(os.path.join(img_dir, f), img_val_dir)

# 레이블 이동
for f in train_img_files:
    label_file = f.replace('.jpg', '.txt')
    shutil.move(os.path.join(label_dir, label_file), label_train_dir)

for f in val_img_files:
    label_file = f.replace('.jpg', '.txt')
    shutil.move(os.path.join(label_dir, label_file), label_val_dir)

print("데이터 분할 및 이동 완료!")

데이터 분할 및 이동 완료!


In [21]:
directory = '/content/yolov8/data/labels/train'
file_count = len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])
print("train dataset:", file_count)

directory = '/content/yolov8/data/labels/val'
file_count = len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])
print("validation dataset:", file_count)

train dataset: 800
validation dataset: 200


# 2. Object Detection

In [22]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [23]:
from ultralytics import YOLO, settings

### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [24]:
settings['datasets_dir'] = '/content/'
settings.update()
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [25]:
# YOLO 학습을 위한 YAML 파일 생성 스크립트

# 필요한 정보 읽기
obj_names_path = '/content/cool_roof_yolo_labels/obj.names'  # 클래스 이름 파일 경로
obj_data_path = '/content/olo_labels/obj.data'  # 데이터셋 설정 파일 경로

# 클래스 이름 로드
with open(obj_names_path, 'r') as f:
    class_names = [line.strip() for line in f.readlines()]

# YAML 파일 내용 작성
yaml_content = f"""
# 데이터셋 경로
path: /content/yolov8/data  # Adjust if necessary
train: /content/yolov8/data/images/train  # 훈련 이미지 경로
val: /content/yolov8/data/images/val  # 검증 이미지 경로

# 클래스 정보
nc: {len(class_names)}  # 클래스 수
names: {class_names}  # 클래스 이름
"""

# YAML 파일 생성
yaml_file = '/content/yolov8/data/dataset.yaml'
with open(yaml_file, 'w') as f:
    f.write(yaml_content.strip())

print(f"{yaml_file}에 YAML 파일이 생성되었습니다.")

/content/yolov8/data/dataset.yaml에 YAML 파일이 생성되었습니다.


### 2) YOLO v8 모델 불러와서 학습

### Nano Model

In [26]:
n_model = YOLO(model='yolov8n.pt', task='detect')

100%|██████████| 6.23M/6.23M [00:00<00:00, 65.3MB/s]


In [27]:
n_model.train(data='/content/yolov8/data/dataset.yaml',
            epochs=100,
            patience=10,
            pretrained=True,
            verbose=True,
            seed=1234,
            )

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/yolov8/data/dataset.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=1234, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

100%|██████████| 755k/755k [00:00<00:00, 17.0MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 18                  -1  1    123648  ultralytics.nn.modules.block.C2f             [192, 128, 1]                 
 19                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 21                  -1  1    493056  ultralytics.nn.modules.block.C2f             [384, 256, 1]                 
 22        [15, 18, 21]  1    751702  ultralytics.nn.modules.head.Detect           [2, [64, 128, 256]]           
Model summary: 225 layers, 3011238 parameters, 3011222 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir runs/detect/train', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/yolov8/data/labels/train... 800 images, 0 backgrounds, 0 corrupt: 100%|██████████| 800/800 [00:01<00:00, 501.44it/s]


...

    100/100      2.34G     0.8153     0.5074      1.065        124        640: 100%|██████████| 50/50 [00:33<00:00,  1.47it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:04<00:00,  1.48it/s]


                   all        200       1559      0.951      0.895       0.97      0.757

100 epochs completed in 1.154 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:10<00:00,  1.56s/it]


                   all        200       1559      0.952      0.895       0.97      0.757
             cool roof        200        227       0.96      0.841      0.957      0.754
          generic roof        200       1332      0.944      0.949      0.982       0.76
Speed: 0.4ms preprocess, 6.3ms inference, 0.0ms loss, 6.2ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a644db07700>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

#### 직접 수집한 인공위성 이미지로 test 해보기

In [28]:
# test 폴더 확인 및 생성 후 test 할 이미지 넣어주기!
dir = 'yolov8/data/test'
os.makedirs(dir, exist_ok=True)

In [29]:
results = n_model.predict(source='/content/yolov8/data/test/*',
                        save=True,
                        line_width=2,
                        stream=True
                         )

In [30]:
for r in results :
    r_bbox = r.boxes


image 1/22 /content/yolov8/data/test/t01.png: 480x640 16 generic roofs, 203.8ms
image 2/22 /content/yolov8/data/test/t02.png: 480x640 12 generic roofs, 18.6ms
image 3/22 /content/yolov8/data/test/t03.png: 576x640 16 generic roofs, 127.0ms
image 4/22 /content/yolov8/data/test/t04.png: 640x640 20 generic roofs, 19.6ms
image 5/22 /content/yolov8/data/test/t05.png: 320x640 8 generic roofs, 119.5ms
image 6/22 /content/yolov8/data/test/t06.png: 640x640 5 generic roofs, 20.9ms
image 7/22 /content/yolov8/data/test/t07.png: 576x640 3 generic roofs, 21.6ms
image 8/22 /content/yolov8/data/test/t08.png: 512x640 15 generic roofs, 130.5ms
image 9/22 /content/yolov8/data/test/test01.png: 480x640 4 generic roofs, 11.7ms
image 10/22 /content/yolov8/data/test/test02.png: 320x640 10 generic roofs, 12.4ms
image 11/22 /content/yolov8/data/test/test03.png: 416x640 13 generic roofs, 62.7ms
image 12/22 /content/yolov8/data/test/test04.png: 640x448 7 generic roofs, 69.3ms
image 13/22 /content/yolov8/data/test